In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
df = pd.read_csv('flipkart_flights_23may.csv')
df.head()

,Airline,Departure,Arrival,Duration,Stop,Price,SearchDate,DepartureDate,Source,Destination,Day,Class
0,IndiGo,16:35,22:00,5h 25m,1 stop - IXC,6734.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹606 off with NEW,e
1,Air India,16:45,23:10,6h 25m,1 stop - IXC,8542.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹885 off with NEW,e
2,IndiGo,16:45,22:25,5h 40m,1 stop - HYD,8727.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹785 off with NEW,e
3,IndiGo,16:25,21:20,4h 55m,1 stop - IDR,9527.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹857 off with NEW,e
4,IndiGo,20:10,01:10,5h,1 stop - NAG,9561.0,2025-05-23 12:00:47,23052025,BOM,DEL,+1D,e


# clearing and processing dataset

In [3]:
df.isnull().sum()
df.dropna(inplace=True)
df.isnull().sum()

Airline          0
Departure        0
Arrival          0
Duration         0
Stop             0
Price            0
SearchDate       0
DepartureDate    0
Source           0
Destination      0
Day              0
Class            0
dtype: int64

In [4]:
# print dublicate value
df.duplicated()
df.drop_duplicates(inplace=True)

In [5]:
def get_total_stop(x):
    if(x.isnumeric()):
            return int(update_str)
    else:
        update_str = x[0]
        if(x == 'Non stop' or update_str == 'u') :
            return 0
        else:
            return int(update_str)

In [6]:
df['total_stop'] = df['Stop'].apply(get_total_stop)

In [7]:
df.head()

,Airline,Departure,Arrival,Duration,Stop,Price,SearchDate,DepartureDate,Source,Destination,Day,Class,total_stop
0,IndiGo,16:35,22:00,5h 25m,1 stop - IXC,6734.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹606 off with NEW,e,1
1,Air India,16:45,23:10,6h 25m,1 stop - IXC,8542.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹885 off with NEW,e,1
2,IndiGo,16:45,22:25,5h 40m,1 stop - HYD,8727.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹785 off with NEW,e,1
3,IndiGo,16:25,21:20,4h 55m,1 stop - IDR,9527.0,2025-05-23 12:00:47,23052025,BOM,DEL,Get ₹857 off with NEW,e,1
4,IndiGo,20:10,01:10,5h,1 stop - NAG,9561.0,2025-05-23 12:00:47,23052025,BOM,DEL,+1D,e,1


In [8]:
df['dept'] = pd.to_datetime(df['DepartureDate'], format='%d%m%Y')
df['SearchDate'] = df['SearchDate'].apply(lambda x :  x.split(" ")[0])


In [9]:
df['SearchDate'] = pd.to_datetime(df['SearchDate'])
df['search_date_day'] = df['SearchDate'].dt.day
df['search_date_month'] = df['SearchDate'].dt.month
df['search_date_year'] = df['SearchDate'].dt.year

In [10]:
df['days_until_departure'] = (df['dept'] - df['SearchDate']).dt.days

In [11]:
df.head(2)

,Airline,Departure,Arrival,Duration,Stop,Price,SearchDate,DepartureDate,Source,Destination,Day,Class,total_stop,dept,search_date_day,search_date_month,search_date_year,days_until_departure
0,IndiGo,16:35,22:00,5h 25m,1 stop - IXC,6734.0,2025-05-23,23052025,BOM,DEL,Get ₹606 off with NEW,e,1,2025-05-23,23,5,2025,0
1,Air India,16:45,23:10,6h 25m,1 stop - IXC,8542.0,2025-05-23,23052025,BOM,DEL,Get ₹885 off with NEW,e,1,2025-05-23,23,5,2025,0


In [12]:
def convert_duration_to_minutes(duration):
    try:
        if not isinstance(duration, str) or duration.strip() == "":
            return 0

        duration = duration.strip().lower()
        days = hours = minutes = 0

        # Extract days
        if 'd' in duration:
            parts = duration.split('d')
            days = int(parts[0].strip())
            duration = parts[1].strip() if len(parts) > 1 else ""

        # Extract hours
        if 'h' in duration:
            parts = duration.split('h')
            hours = int(parts[0].strip())
            duration = parts[1].strip() if len(parts) > 1 else ""

        # Extract minutes
        if 'm' in duration:
            minutes = int(duration.replace('m', '').strip())

        return days * 1440 + hours * 60 + minutes
    except Exception as e:
        print(f"Warning: Failed to parse duration '{duration}' – {e}")
        return 0

In [13]:
convert_duration_to_minutes('2h 3m')

123

In [14]:
# Convert duration to total minutes
df['duration_mins'] = df['Duration'].apply(convert_duration_to_minutes)

In [15]:
from datetime import datetime

# Define festival date ranges (customize based on year and known dates)
festival_seasons = [
    ('Diwali', datetime(2025, 10, 20), datetime(2025, 10, 26)),
    ('Dussehra', datetime(2025, 10, 1), datetime(2025, 10, 5)),
    ('Christmas', datetime(2025, 12, 24), datetime(2025, 12, 26)),
    ('Holi', datetime(2025, 3, 13), datetime(2025, 3, 14)),
    ('Independence Day', datetime(2025, 8, 15), datetime(2025, 8, 15)),
    ('New Year', datetime(2025, 12, 31), datetime(2026, 1, 1))
]

def check_festival_season(date_to_check):
    for festival_name, start_date, end_date in festival_seasons:
        if start_date <= date_to_check <= end_date:
            return 1
    return 0

# Example usage
test_date = datetime(2025, 10, 22)
print(test_date)
print(check_festival_season(test_date))

2025-10-22 00:00:00
1


In [16]:
df['festival_season'] = df['dept'].apply(check_festival_season)

In [17]:
df.tail(4)

,Airline,Departure,Arrival,Duration,Stop,Price,SearchDate,DepartureDate,Source,Destination,Day,Class,total_stop,dept,search_date_day,search_date_month,search_date_year,days_until_departure,duration_mins,festival_season
582204,Air India,10:10,14:50,4h 40m,1 stop - BOM,50956.0,2025-06-07,30052026,COK,GOI,Get ₹900 off with FKCCDC,b,1,2026-05-30,7,6,2025,357,280,0
582205,Air India,07:00,14:15,7h 15m,1 stop - DEL,68290.0,2025-06-07,30052026,COK,GOI,Get ₹900 off with FKCCDC,b,1,2026-05-30,7,6,2025,357,435,0
582206,Air India,12:00,19:00,7h,1 stop - DEL,68750.0,2025-06-07,30052026,COK,GOI,Get ₹900 off with FKCCDC,b,1,2026-05-30,7,6,2025,357,420,0
582207,Air India,15:20,00:20,9h,1 stop - DEL,68750.0,2025-06-07,30052026,COK,GOI,+1D,b,1,2026-05-30,7,6,2025,357,540,0


In [18]:
df['dept_day_of_week'] = df['dept'].dt.dayofweek
df['dept_day_of_year'] = df['dept'].dt.dayofyear
df['dept_is_weekend'] = df['dept'].isin([5, 6]).astype(int)

In [19]:
# Function to classify time segments
def get_time_segment(time_data):
    hour = int(time_data.split(':')[0].strip())
    if 0 <= hour < 5:
        return "Late_Night"
    elif 5 <= hour < 8:
        return "Early_Morning"
    elif 8 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

# Apply function to DataFrame

In [20]:
# Apply function to DataFrame column
df["departure_segment"] = df["Departure"].apply(get_time_segment)
df["arrival_segment"] = df["Arrival"].apply(get_time_segment)

In [21]:
# find outlier
Q1 = df["Price"].quantile(0.25)
Q3 = df["Price"].quantile(0.75)
iqr = Q3 - Q1
maxnum = Q3 + 1.5 * iqr
minnum = Q1 - 1.5 * iqr
print(maxnum)
print(minnum)

56388.75
-22641.25


In [22]:
np.where(df["Price"] > maxnum, df["Price"].median(), df["Price"])

array([ 6734.,  8542.,  8727., ..., 12919., 12919., 12919.])

In [23]:
df.head()

,Airline,Departure,Arrival,Duration,Stop,Price,SearchDate,DepartureDate,Source,Destination,...,search_date_month,search_date_year,days_until_departure,duration_mins,festival_season,dept_day_of_week,dept_day_of_year,dept_is_weekend,departure_segment,arrival_segment
0,IndiGo,16:35,22:00,5h 25m,1 stop - IXC,6734.0,2025-05-23,23052025,BOM,DEL,...,5,2025,0,325,0,4,143,0,Afternoon,Night
1,Air India,16:45,23:10,6h 25m,1 stop - IXC,8542.0,2025-05-23,23052025,BOM,DEL,...,5,2025,0,385,0,4,143,0,Afternoon,Night
2,IndiGo,16:45,22:25,5h 40m,1 stop - HYD,8727.0,2025-05-23,23052025,BOM,DEL,...,5,2025,0,340,0,4,143,0,Afternoon,Night
3,IndiGo,16:25,21:20,4h 55m,1 stop - IDR,9527.0,2025-05-23,23052025,BOM,DEL,...,5,2025,0,295,0,4,143,0,Afternoon,Night
4,IndiGo,20:10,01:10,5h,1 stop - NAG,9561.0,2025-05-23,23052025,BOM,DEL,...,5,2025,0,300,0,4,143,0,Evening,Late_Night


In [24]:
df.drop('Stop', axis=1, inplace=True)
df.drop('Duration', axis=1, inplace=True)

In [25]:
df.drop('Day', axis=1, inplace=True)

In [26]:
df.columns

Index(['Airline', 'Departure', 'Arrival', 'Price', 'SearchDate',
       'DepartureDate', 'Source', 'Destination', 'Class', 'total_stop', 'dept',
       'search_date_day', 'search_date_month', 'search_date_year',
       'days_until_departure', 'duration_mins', 'festival_season',
       'dept_day_of_week', 'dept_day_of_year', 'dept_is_weekend',
       'departure_segment', 'arrival_segment'],
      dtype='object')

In [27]:
### scale the price for make price in range

In [28]:
x_min = df['Price'].min()
x_max = df['Price'].max()
def x_in_range(x):
   return (x - x_min) / (x_max - x_min)
df['Price'] = df['Price'].apply(x_in_range)

In [29]:
df.head()

,Airline,Departure,Arrival,Price,SearchDate,DepartureDate,Source,Destination,Class,total_stop,...,search_date_month,search_date_year,days_until_departure,duration_mins,festival_season,dept_day_of_week,dept_day_of_year,dept_is_weekend,departure_segment,arrival_segment
0,IndiGo,16:35,22:00,0.034592,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,325,0,4,143,0,Afternoon,Night
1,Air India,16:45,23:10,0.047722,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,385,0,4,143,0,Afternoon,Night
2,IndiGo,16:45,22:25,0.049066,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,340,0,4,143,0,Afternoon,Night
3,IndiGo,16:25,21:20,0.054876,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,295,0,4,143,0,Afternoon,Night
4,IndiGo,20:10,01:10,0.055123,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,300,0,4,143,0,Evening,Late_Night


In [30]:
df.head()

,Airline,Departure,Arrival,Price,SearchDate,DepartureDate,Source,Destination,Class,total_stop,...,search_date_month,search_date_year,days_until_departure,duration_mins,festival_season,dept_day_of_week,dept_day_of_year,dept_is_weekend,departure_segment,arrival_segment
0,IndiGo,16:35,22:00,0.034592,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,325,0,4,143,0,Afternoon,Night
1,Air India,16:45,23:10,0.047722,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,385,0,4,143,0,Afternoon,Night
2,IndiGo,16:45,22:25,0.049066,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,340,0,4,143,0,Afternoon,Night
3,IndiGo,16:25,21:20,0.054876,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,295,0,4,143,0,Afternoon,Night
4,IndiGo,20:10,01:10,0.055123,2025-05-23,23052025,BOM,DEL,e,1,...,5,2025,0,300,0,4,143,0,Evening,Late_Night


## Label encoding of airport name

In [31]:
from sklearn.preprocessing import LabelEncoder

In [32]:
label_encoder = LabelEncoder()
df['Source'] = label_encoder.fit_transform(df['Source'])

In [33]:
label_encoder = LabelEncoder()
df['Destination'] = label_encoder.fit_transform(df['Destination'])
df['Airline'] = label_encoder.fit_transform(df['Airline'])
df['Class'] = label_encoder.fit_transform(df['Class'])


In [34]:
df['departure_segment'] = label_encoder.fit_transform(df['departure_segment'])
df['arrival_segment'] = label_encoder.fit_transform(df['arrival_segment'])
df['Departure'] = label_encoder.fit_transform(df['Departure'])
df['Arrival'] = label_encoder.fit_transform(df['Arrival'])
df['SearchDate'] = label_encoder.fit_transform(df['SearchDate'])
df['DepartureDate'] = label_encoder.fit_transform(df['DepartureDate'])

In [35]:
df.head()

,Airline,Departure,Arrival,Price,SearchDate,DepartureDate,Source,Destination,Class,total_stop,...,search_date_month,search_date_year,days_until_departure,duration_mins,festival_season,dept_day_of_week,dept_day_of_year,dept_is_weekend,departure_segment,arrival_segment
0,5,193,264,0.034592,0,83,2,5,1,1,...,5,2025,0,325,0,4,143,0,0,5
1,0,195,278,0.047722,0,83,2,5,1,1,...,5,2025,0,385,0,4,143,0,0,5
2,5,195,269,0.049066,0,83,2,5,1,1,...,5,2025,0,340,0,4,143,0,0,5
3,5,191,256,0.054876,0,83,2,5,1,1,...,5,2025,0,295,0,4,143,0,0,5
4,5,236,15,0.055123,0,83,2,5,1,1,...,5,2025,0,300,0,4,143,0,2,3


In [36]:
# check data type
df.dtypes

Airline                          int64
Departure                        int64
Arrival                          int64
Price                          float64
SearchDate                       int64
DepartureDate                    int64
Source                           int64
Destination                      int64
Class                            int64
total_stop                       int64
dept                    datetime64[ns]
search_date_day                  int32
search_date_month                int32
search_date_year                 int32
days_until_departure             int64
duration_mins                    int64
festival_season                  int64
dept_day_of_week                 int32
dept_day_of_year                 int32
dept_is_weekend                  int64
departure_segment                int64
arrival_segment                  int64
dtype: object

In [37]:
df['dept']  = df['dept'].astype(int)

In [38]:
y = df['Price']
df_copy = df.copy()
df_copy.drop('Price', axis=1, inplace=True)
X = df_copy.copy()

In [39]:
X.shape

(582115, 21)

In [40]:
y.shape

(582115,)

In [41]:
# check any null value
X.isnull

<bound method DataFrame.isnull of         Airline  Departure  Arrival  SearchDate  DepartureDate  Source  \
0             5        193      264           0             83       2   
1             0        195      278           0             83       2   
2             5        195      269           0             83       2   
3             5        191      256           0             83       2   
4             5        236       15           0             83       2   
...         ...        ...      ...         ...            ...     ...   
582203        0        178        5          11            124       4   
582204        0        116      177          11            124       4   
582205        0         78      170          11            124       4   
582206        0        138      228          11            124       4   
582207        0        178        5          11            124       4   

        Destination  Class  total_stop                 dept  ...  \
0        

### Preprocessing

In [42]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate,GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

2025-06-08 21:44:38.996402: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749419079.347992      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749419079.476401      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [43]:
X.drop('dept', axis=1, inplace=True)

In [44]:
X.drop('Departure', axis=1, inplace=True)
X.drop('Arrival', axis=1, inplace=True)

In [45]:

X.drop('DepartureDate', axis=1, inplace=True)

In [46]:
X.drop('SearchDate', axis=1, inplace=True)

In [47]:
X.columns

Index(['Airline', 'Source', 'Destination', 'Class', 'total_stop',
       'search_date_day', 'search_date_month', 'search_date_year',
       'days_until_departure', 'duration_mins', 'festival_season',
       'dept_day_of_week', 'dept_day_of_year', 'dept_is_weekend',
       'departure_segment', 'arrival_segment'],
      dtype='object')

In [48]:
df.head()

,Airline,Departure,Arrival,Price,SearchDate,DepartureDate,Source,Destination,Class,total_stop,...,search_date_month,search_date_year,days_until_departure,duration_mins,festival_season,dept_day_of_week,dept_day_of_year,dept_is_weekend,departure_segment,arrival_segment
0,5,193,264,0.034592,0,83,2,5,1,1,...,5,2025,0,325,0,4,143,0,0,5
1,0,195,278,0.047722,0,83,2,5,1,1,...,5,2025,0,385,0,4,143,0,0,5
2,5,195,269,0.049066,0,83,2,5,1,1,...,5,2025,0,340,0,4,143,0,0,5
3,5,191,256,0.054876,0,83,2,5,1,1,...,5,2025,0,295,0,4,143,0,0,5
4,5,236,15,0.055123,0,83,2,5,1,1,...,5,2025,0,300,0,4,143,0,2,3


In [49]:
df_copy.head()

,Airline,Departure,Arrival,SearchDate,DepartureDate,Source,Destination,Class,total_stop,dept,...,search_date_month,search_date_year,days_until_departure,duration_mins,festival_season,dept_day_of_week,dept_day_of_year,dept_is_weekend,departure_segment,arrival_segment
0,5,193,264,0,83,2,5,1,1,1747958400000000000,...,5,2025,0,325,0,4,143,0,0,5
1,0,195,278,0,83,2,5,1,1,1747958400000000000,...,5,2025,0,385,0,4,143,0,0,5
2,5,195,269,0,83,2,5,1,1,1747958400000000000,...,5,2025,0,340,0,4,143,0,0,5
3,5,191,256,0,83,2,5,1,1,1747958400000000000,...,5,2025,0,295,0,4,143,0,0,5
4,5,236,15,0,83,2,5,1,1,1747958400000000000,...,5,2025,0,300,0,4,143,0,2,3


## Start Multi-Layer Perceptron

In [50]:
print(X.shape)
print(y.shape)

(582115, 16)
(582115,)


In [51]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [52]:
X_scaled = scaler.fit_transform(X)

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model_nlp = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # regression output
])

model_nlp.compile(optimizer=Adam(0.0005), loss='mse', metrics=['mae'])
model_nlp.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-08 21:44:58.716743: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           2,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,545 (49.00 KB)

 Trainable params: 12,545 (49.00 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
# history_nlp = model_nlp.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=64)

In [56]:
# import matplotlib.pyplot as plt

# # Plotting Training and Validation Loss
# plt.plot(history_nlp.history['loss'], label='Training Loss')
# plt.plot(history_nlp.history['val_loss'], label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()


In [57]:
# -------------------------
# 1. Select Features
# -------------------------
seq_features = ['days_until_departure', 'duration_mins', 'total_stop',
                'dept_day_of_week', 'dept_day_of_year', 'dept_is_weekend',
                'search_date_day', 'search_date_month', 'search_date_year',
                'festival_season']

In [58]:
static_cat_features = ['Airline','Source', 'Destination', 'Class']
target = 'Price'

In [59]:
# -------------------------
# 2. Normalize sequential features
# -------------------------
scaler = MinMaxScaler()
df[seq_features] = scaler.fit_transform(df[seq_features])

In [60]:
# -------------------------
# 3. Encode categorical (static) features
# -------------------------
df_static = pd.get_dummies(df[static_cat_features])


In [ ]:
df.head()

In [61]:
# -------------------------
# 4. Create Sequences for LSTM (reshape to 3D)
# -------------------------
X_seq = df[seq_features].values.reshape(-1, 1, len(seq_features))  # (N, 1, D)
X_static = df_static.values  # (N, D2)
y = df[target].values  # (N,)


In [62]:
# -------------------------
# 5. Train/Test Split
# -------------------------
X_seq_train, X_seq_test, X_static_train, X_static_test, y_train, y_test = train_test_split(
    X_seq, X_static, y, test_size=0.2, random_state=42)

In [63]:
#4. Build the LSTM Model with Mixed Inputs

In [64]:
# Sequential Input
seq_input = Input(shape=(1, len(seq_features)))
lstm_out = LSTM(64)(seq_input)

# Static Input
static_input = Input(shape=(X_static.shape[1],))

# Merge and Dense Layers
merged = Concatenate()([lstm_out, static_input])
dense1 = Dense(64, activation='relu')(merged)
dense2 = Dense(32, activation='relu')(dense1)
output = Dense(1)(dense2)

model = Model(inputs=[seq_input, static_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 1, 10)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 64)             │         19,200 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 68)             │              0 │ lstm[0][0],            │
│                           │                        │                │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │          4,416 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 32)             │          2,080 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 1)              │             33 │ dense_5[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 25,729 (100.50 KB)

 Trainable params: 25,729 (100.50 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [66]:
# history = model.fit([X_seq_train, X_static_train], y_train,
#           validation_split=0.3, epochs=100, batch_size=64, callbacks=[early_stop])

In [67]:
loss, mae = model.evaluate([X_seq_test, X_static_test], y_test)
print(f"Test MAE: {mae}")


3639/3639 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 0.1143 - mae: 0.2849
Test MAE: 0.2858274281024933


In [68]:
# import matplotlib.pyplot as plt

# # Plotting Training and Validation Loss
# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Training and Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()


### Apply GRU method

In [69]:
# Sequential Input
seq_input = Input(shape=(1, len(seq_features)))
lstm_out = GRU(64)(seq_input)

# Static Input
static_input = Input(shape=(X_static.shape[1],))

# Merge and Dense Layers
merged = Concatenate()([lstm_out, static_input])
dense1 = Dense(64, activation='relu')(merged)
dense2 = Dense(32, activation='relu')(dense1)
output = Dense(1)(dense2)

model = Model(inputs=[seq_input, static_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0005), loss='mse', metrics=['mae'])
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 1, 10)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ gru (GRU)                 │ (None, 64)             │         14,592 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 68)             │              0 │ gru[0][0],             │
│ (Concatenate)             │                        │                │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 64)             │          4,416 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 32)             │          2,080 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 1)              │             33 │ dense_8[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,121 (82.50 KB)

 Trainable params: 21,121 (82.50 KB)

 Non-trainable params: 0 (0.00 B)

In [70]:
history_gru = model.fit([X_seq_train, X_static_train], y_train,
          validation_split=0.3, epochs=100, batch_size=64, callbacks=[early_stop])

Epoch 1/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 33s 6ms/step - loss: 0.0112 - mae: 0.0545 - val_loss: 0.0025 - val_mae: 0.0323
Epoch 2/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 26s 5ms/step - loss: 0.0026 - mae: 0.0324 - val_loss: 0.0029 - val_mae: 0.0365
Epoch 3/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 30s 6ms/step - loss: 0.0025 - mae: 0.0313 - val_loss: 0.0024 - val_mae: 0.0296
Epoch 4/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 30s 6ms/step - loss: 0.0024 - mae: 0.0306 - val_loss: 0.0024 - val_mae: 0.0294
Epoch 5/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 28s 5ms/step - loss: 0.0022 - mae: 0.0295 - val_loss: 0.0022 - val_mae: 0.0286
Epoch 6/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - loss: 0.0022 - mae: 0.0290 - val_loss: 0.0021 - val_mae: 0.0270
Epoch 7/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - loss: 0.0021 - mae: 0.0283 - val_loss: 0.0020 - val_mae: 0.0273
Epoch 8/100
5094/5094 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - loss: 0.0020 - mae: 0.0281 - val_loss: 0.0021 - val_mae: 0.0283
Epoch 9/100
5094/5094 ━━

In [71]:
# Plotting Training and Validation Loss
plt.plot(history_gru.history['loss'], label='Training Loss')
plt.plot(history_gru.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Plotting validation loss three model
# plt.plot(history_gru.history['loss'], label='Validation GRU Loss')
# plt.plot(history_nlp.history['loss'], label='Validation MLP Loss')
# plt.plot(history.history['loss'], label='Validation LSTM Loss')
# plt.title('Validation Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()